In [1]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('tagsets')
nltk.download('stopwords')
!pip install wikipedia
import wikipedia
from sklearn.feature_extraction.text import TfidfVectorizer
import string

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp36-none-any.whl size=11686 sha256=f193d511405b2f2881d59b0b4cad86fc8c49d8b32a081e72d286804c4d139d64
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [0]:
import json
with open("data.json", "r") as read_file:
    data = json.load(read_file)

In [0]:
with open("infobox.json", "r") as read_file:
    infobox_data = json.load(read_file)

In [0]:
from nltk.corpus import stopwords

In [0]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from pprint import pprint

In [44]:
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [0]:
import en_core_web_lg
nlp = en_core_web_lg.load()

# Preprocess Wikipedia Sentences

NER using Spacy: https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da

Basic NLP Overview: https://github.com/charlieg/A-Smattering-of-NLP-in-Python

Stackoverflow Q about Info Extraction: https://datascience.stackexchange.com/questions/2646/extract-most-informative-parts-of-text-from-documents

## TODO: Read these and try them out:

Context analysis for NLP: https://www.lexalytics.com/lexablog/context-analysis-nlp

Using lexical chains for keyword extraction: https://web.cs.hacettepe.edu.tr/~ilyas/PDF/2007_IPM.pdf

Event Detection using Lexical Chains: https://link.springer.com/chapter/10.1007/978-3-642-14770-8_35

Using Lexical Chains for Text Summarization: https://www.aclweb.org/anthology/W97-0703.pdf

Significant Lexical Relationships: https://www.d.umn.edu/~tpederse/Pubs/aaai96-cmpl.pdf





In [0]:
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [0]:
samp_sent = ("The university is located in San Luis Obispo, California, often noted "
            "as one of the happiest cities in the United States, with many alumni in Silicon Valley.")
samp_para = wikipedia.page('california polytechnic school').section('History').split("\n")[0]

## Spacy NLP

In [0]:
query_nlp = nlp("What is the university population?")

In [21]:
list(samp_pop_nlp.noun_chunks)

NameError: ignored

In [0]:
samp_pop_nlp = nlp("The Cal Poly Master Plan calls to increase student population from approximately 17,000 students to 20,900 students by the year 2020–2021")

In [23]:
' '.join([str(word) for word in samp_pop_nlp.noun_chunks])

'The Cal Poly Master Plan student population approximately 17,000 students 20,900 students the year'

In [24]:
' '.join([str(word) for word in query_nlp.noun_chunks])

'What the university population'

In [25]:
nlp(' '.join([str(word) for word in query_nlp.noun_chunks])).similarity(nlp(' '.join([str(word) for word in samp_pop_nlp.noun_chunks])))

0.8062704332689763

In [26]:
for word in query_nlp:
    print(word, word.pos_)

What PRON
is VERB
the DET
university NOUN
population NOUN
? PUNCT


In [0]:
infobox_similarity = {}
for info_name in infobox_data.keys():
    infobox_row = info_name + " "
    if isinstance(infobox_data[info_name], str):
        infobox_row += infobox_data[info_name]
    else:
        infobox_row += ' '.join(infobox_data[info_name])
    infobox_similarity[infobox_row] = nlp(infobox_row).similarity(query_nlp)


In [0]:
section_similarity = {}
for para in wikipedia.page('california polytechnic school').content.split("\n"):
    for sent in nltk.sent_tokenize(para):
        section_similarity[sent] = nlp(sent).similarity(query_nlp)

In [29]:
pprint(sorted(section_similarity.items(), key = 
             lambda kv:(kv[1], kv[0]), reverse=True))

[('As of 2018, Cal Poly San Luis Obispo has the least racially diverse student '
  'population of all California State University and University of California '
  'campuses, though it is still far more diverse than the surrounding '
  'community it serves.',
  0.894135331859435),
 ('It is one of two polytechnics in the California State University system.',
  0.8786171526754405),
 ('This ROI is the 2nd highest in the California State University system and '
  'is higher than all of the University of California schools except UC '
  'Berkeley.Washington Monthly ranked Cal Poly 23rd out of 606 schools in the '
  'U.S. "National Universities - Masters" category in 2019 based on its '
  'contribution to the public good in three broad categories: social mobility, '
  'research, and service.',
  0.8783312631472701),
 ("The state legislature changed the school's official name again in 1971 to "
  'California Polytechnic State University, and since the 1970s the university '
  'has seen steady 

In [30]:
pprint(sorted(infobox_similarity.items(), key = 
             lambda kv:(kv[1], kv[0]), reverse=True))

[('type Public University and Space-grant university', 0.7948758349361857),
 ('affiliations California State University | American Association of State '
  'Colleges and Universities | Association of Public and Land-Grant '
  'Universities',
  0.765226243033916),
 ('country United States', 0.6747147599404637),
 ('name California Polytechnic State University', 0.6683719481013191),
 ('english motto Learn by Doing', 0.6645101867717298),
 ('campus Suburban, 9,678 acres (3,917 ha) Total; 1,321 acres (535 ha) for the '
  'Main Campus; 155 acres (63 ha) for the Campus Core',
  0.648299745361463),
 ('athletics National Collegiate Athletic Association | NCAA Division I | '
  'Football Championship Subdivision | Big West Conference',
  0.5928908104369979),
 ('state California', 0.5517238985862338),
 ('students 21,242 (Fall 2019)', 0.5379765324216425),
 ('mascots Chase the Mustang, Musty the Mustang', 0.5309994455214994),
 ('colors Green and Gold', 0.508922575680383),
 ('established March 8, 1901

### Named Entity Recognition

In [45]:
doc = nlp(samp_sent)
pprint([(X.text, X.label_) for X in doc.ents])

[('San Luis Obispo', 'GPE'),
 ('California', 'GPE'),
 ('one', 'CARDINAL'),
 ('the United States', 'GPE'),
 ('Silicon Valley', 'LOC')]


In [46]:
displacy.render(nlp(samp_sent), jupyter=True, style='ent')

In [47]:
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in doc 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]

[('university', 'NOUN', 'university'),
 ('located', 'VERB', 'locate'),
 ('San', 'PROPN', 'San'),
 ('Luis', 'PROPN', 'Luis'),
 ('Obispo', 'PROPN', 'Obispo'),
 ('California', 'PROPN', 'California'),
 ('noted', 'VERB', 'note'),
 ('happiest', 'ADJ', 'happy'),
 ('cities', 'NOUN', 'city'),
 ('United', 'PROPN', 'United'),
 ('States', 'PROPN', 'States'),
 ('alumni', 'NOUN', 'alumnus'),
 ('Silicon', 'PROPN', 'Silicon'),
 ('Valley', 'PROPN', 'Valley')]

In [48]:
dict([(str(x), x.label_) for x in doc.ents])


{'California': 'GPE',
 'San Luis Obispo': 'GPE',
 'Silicon Valley': 'LOC',
 'one': 'CARDINAL',
 'the United States': 'GPE'}

### TF-IDF

In [0]:
# Computes TF for a sentence
def computeTF(sent):
    words = nltk.word_tokenize(sent)
    word_count = {}
    for word in words:
        if word in stopwords.words('english'):
            continue
        if word in string.punctuation:
            continue
        if word not in word_count.keys():
            word_count[word] = 1.0
        else:
            word_count[word] += 1.0
    for word in word_count.keys():
        word_count[word]/= len(words)
    return sorted(word_count.items(), key = 
             lambda kv:(kv[1], kv[0]), reverse=True)

In [50]:
computeTF(samp_sent)

[('university', 0.03225806451612903),
 ('one', 0.03225806451612903),
 ('often', 0.03225806451612903),
 ('noted', 0.03225806451612903),
 ('many', 0.03225806451612903),
 ('located', 0.03225806451612903),
 ('happiest', 0.03225806451612903),
 ('cities', 0.03225806451612903),
 ('alumni', 0.03225806451612903),
 ('Valley', 0.03225806451612903),
 ('United', 0.03225806451612903),
 ('The', 0.03225806451612903),
 ('States', 0.03225806451612903),
 ('Silicon', 0.03225806451612903),
 ('San', 0.03225806451612903),
 ('Obispo', 0.03225806451612903),
 ('Luis', 0.03225806451612903),
 ('California', 0.03225806451612903)]

#### TF-IDF for paragraphs

In [51]:
import datetime, re, sys
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer

tokens = [word for sent in nltk.sent_tokenize(wikipedia.page('california polytechnic school').section('History')) for word in nltk.word_tokenize(sent)]

stemmer = SnowballStemmer("english")
stemmed_tokens = [stemmer.stem(t) for t in tokens]

def tokenize_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

token_dict = {}
split_wikipedia = wikipedia.page('california polytechnic school').section('History').split("\n")
for i,sent in enumerate(wikipedia.page('california polytechnic school').section('History').split("\n")):
    token_dict[str(i)] = sent
        
tfidf = TfidfVectorizer(tokenizer=tokenize_and_stem, stop_words='english', decode_error='ignore')
print("building term-document matrix... [process started: " + str(datetime.datetime.now()) + ']')
sys.stdout.flush()

tdm = tfidf.fit_transform(token_dict.values()) # this can take some time (about 60 seconds on my machine)
print('done! [process finished: ' + str(datetime.datetime.now()) + ']')

building term-document matrix... [process started: 2020-03-10 22:57:54.331488]
done! [process finished: 2020-03-10 22:57:54.383931]


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [52]:
from random import randint

feature_names = tfidf.get_feature_names()
print('TDM contains ' + str(len(feature_names)) + ' terms and ' + str(tdm.shape[0]) + ' documents')

print('first term: ' + feature_names[0])
print('last term: ' + feature_names[len(feature_names) - 1])

for i in range(0, 4):
    print('random term: ' + feature_names[randint(1,len(feature_names) - 2)])

TDM contains 129 terms and 4 documents
first term: 's
last term: year
random term: bachelor
random term: cal
random term: institut
random term: agricultur


In [53]:
import math
from __future__ import division

article_id = 0
article_text = wikipedia.page('california polytechnic school').section('History').split("\n")[0]

sent_scores = []
for sentence in nltk.sent_tokenize(article_text):
    score = 0
    sent_tokens = tokenize_and_stem(sentence)
    for token in (t for t in sent_tokens if t in feature_names):
        score += tdm[article_id, feature_names.index(token)]
    sent_scores.append((score / len(sent_tokens), sentence))

summary_length = int(math.ceil(len(sent_scores) / 5))
sent_scores.sort(key=lambda sent: sent[0])

print('*** SUMMARY ***')
for summary_sentence in sent_scores[:summary_length]:
    pprint(summary_sentence[1])

print('\n*** ORIGINAL ***')
pprint(article_text)

*** SUMMARY ***
('The school continued to grow steadily, except during a period from the mid '
 '1910s to the early 1920s when World War I led to drops in enrollment and '
 'drastic budget cuts forced fewer class offerings.')

*** ORIGINAL ***
('Cal Poly San Luis Obispo was established as the California Polytechnic '
 'School in 1901 when Governor Henry T. Gage signed the California Polytechnic '
 'School Bill after a campaign by journalist Myron Angel. The polytechnic '
 'school held its first classes on October 1, 1903 to 20 students, offering '
 'secondary level courses of study, which took three years to complete. The '
 'school continued to grow steadily, except during a period from the mid 1910s '
 'to the early 1920s when World War I led to drops in enrollment and drastic '
 'budget cuts forced fewer class offerings.')


## Result: Function to generate important info per sentence
- Noun Chunking
- Named Entity Recognition
- POS Tagging

In [0]:
# Computes TF for a sentence
def computeTF(sent):
    words = nltk.word_tokenize(sent)
    word_count = {}
    for word in words:
        if word in stopwords.words('english'):
            continue
        if word in string.punctuation:
            continue
        if word not in word_count.keys():
            word_count[word] = 1.0
        else:
            word_count[word] += 1.0
    for word in word_count.keys():
        word_count[word]/= len(words)
    return sorted(word_count.items(), key = 
             lambda kv:(kv[1], kv[0]), reverse=True)

In [0]:
# Given a sentence, return it's important information
#   - Noun Chunking
#   - Named Entity Recognition
#   - POS Tagging
#   - Term Frequency
#   - Raw Text 

def generate_info(sent):
    returned_data = {}
    doc = nlp(sent)
    returned_data['TF'] = computeTF(sent)
    returned_data['nn_chunking'] = [str(word) for word in doc.noun_chunks]
    returned_data['ner'] = {str(x):x.label_ for x in doc.ents}
    returned_data['pos'] = [(str(word), word.pos_) for word in doc]
    returned_data['raw'] = sent
    return returned_data

In [0]:
for section in data:
    sentences = nltk.sent_tokenize(data[section]['content'])
    data[section]['parsed_sent'] = {}
    for i, sent in enumerate(sentences):
        data[section]['parsed_sent'][str(i)] = generate_info(sent)

In [0]:
with open('data_wInfo.json', 'w') as f:
    json.dump(data, f)

# Understanding Queries

May be useful: https://towardsdatascience.com/how-to-create-your-own-question-answering-system-easily-with-python-2ef8abc8eb5

Paper on using Wiki to answer questions: https://arxiv.org/pdf/1704.00051.pdf

In [0]:
query = "Is Cal Poly a school?"

In [0]:
nltk.pos_tag(nltk.word_tokenize(query))

[('Is', 'VBZ'),
 ('Cal', 'NNP'),
 ('Poly', 'NNP'),
 ('a', 'DT'),
 ('school', 'NN'),
 ('?', '.')]

In [0]:
# nltk.help.upenn_tagset()